In [2]:
!pip install openai

In [12]:
from openai import OpenAI
import numpy as np
import pandas as pd
import os

from google.colab import userdata
api_key = userdata.get('sutaaka')

if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("APIキーが見つかりません。")

client = OpenAI()

In [4]:
def vectorize_texts(text):
    """ 文書リストをベクトル化して返します """
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return np.array(response.data[0].embedding).reshape(1, -1)

In [5]:
def find_most_similar(question_vector, vectors, documents):
    """ 類似度が最も高い文書を見つけます """
    vectors = np.vstack(vectors)  # ドキュメントベクトルを垂直に積み重ねて2次元配列にする
    similarities = np.dot(vectors, question_vector.T).flatten() / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(question_vector))
    print("similarities", similarities)
    most_similar_index = np.argmax(similarities)
    return documents[most_similar_index]

In [6]:
def ask_question(question, context):
    """ GPTを使って質問に答えます """
    prompt = f'''以下の質問に以下の情報をベースにして答えてください。
            [ユーザーの質問]
            {question}

            [情報]
            {context}
            '''
    print("prompt", prompt)
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=200
    )
    return response.choices[0].text

In [7]:
# 情報
documents = [
    "2023年上期売上200億円、下期売上300億円",
    "2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]

# 文書をベクトル化
vectors = [vectorize_texts(doc) for doc in documents]

# 質問
question = "2023年の第1事業部の売上ってどのくらい？"

# 質問をベクトル化
question_vector = vectorize_texts(question)

# 最も類似した文書を見つける
similar_document = find_most_similar(question_vector, vectors, documents)

# GPTモデルに質問
answer = ask_question(question, similar_document)
print(answer)

similarities [0.60348213 0.85603513 0.62467734]
prompt 以下の質問に以下の情報をベースにして答えてください。
            [ユーザーの質問]
            2023年の第1事業部の売上ってどのくらい？

            [情報]
            2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円
            

2023年の第1事業部の売上は300億円です。


In [1]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity


url = "https://toukei-lab.com/achademy/?page_id=1619"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

text_nodes = soup.find_all("div")

joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)

In [8]:
print(joined_text)

MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 株価予測コンペに挑戦してAmazonギフト券をゲットしよう！【11月30日まで】AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月に1コース追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォームでの24時間以内の返信対応（カリキュラムが終了するまでずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）含む現役データサイエンティストとのマンツーマンメンタリング（4回分）・全

In [9]:
chunk_size = 400
overlap = 50
chunks = []
start = 0

while start + chunk_size <= len(joined_text):
  chunks.append(joined_text[start:start + chunk_size])
  start += (chunk_size - overlap)

if start < len(joined_text):
  chunks.append(joined_text[-chunk_size:])

In [10]:
print(chunks)

['MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 株価予測コンペに挑戦してAmazonギフト券をゲットしよう！【11月30日まで】AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プ', 'ログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月に1コース追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミア', '付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォーム

In [13]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('sutaaka')
client = OpenAI()


def scrape_article(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  text_nodes = soup.find_all("div")
  joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)

  return joined_text


def chunk_text(text, chunk_size, overlap):
  chunks = []
  start = 0
  while start + chunk_size <= len(text):
    chunks.append(text[start:start + chunk_size])
    start += (chunk_size - overlap)
  if start < len(text):
    chunks.append(text[-chunk_size:])

  return chunks


def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding


def find_most_similar(question_vector, vectors, documents):
  similarities = []

  for index, vector in enumerate(vectors):
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    similarities.append([similarity, index])

  similarities.sort(reverse=True, key=lambda x: x[0])
  top_documents = [documents[index] for similarity, index in similarities[:2]]

  return top_documents


def ask_question(question, context):
  prompt = f'''以下の質問に以下の情報をベースにして答えて下さい。
  [ユーザーの質問]
  {question}

  [情報]
  {context}
  '''
  print(prompt)
  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=200
  )

  return response.choices[0].text


url = "https://toukei-lab.com/achademy/?page_id=1619"
chunk_size = 400
overlap = 50

article_text = scrape_article(url)
text_chunks = chunk_text(article_text, chunk_size, overlap)

vectors = [vectorize_text(doc) for doc in text_chunks]

question = "オーダーメイドプランの価格はいくらですか？"
question_vector = vectorize_text(question)

similar_document = find_most_similar(question_vector, vectors, text_chunks)

answer = ask_question(question, similar_document)
print(answer)

以下の質問に以下の情報をベースにして答えて下さい。
  [ユーザーの質問]
  オーダーメイドプランの価格はいくらですか？

  [情報]
  ['てみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラ', '参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみる試してみるオーダーメイドプランは一人一人の目指す姿を実現できるように徹底的に伴走させていただくプランです・スタアカ専用QAフォームでの24時間以内の返信対応（カリキュラムが終了するまでずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）とのマンツーマンメンタリング（月1回×8ヶ月）・前半4ヶ月でスタアカカリキュラムを終わらせ残りの4ヶ月でオーダーメイドの学習・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics1012）のUdemyコース5種類の贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイ']
  
 
  私の理解では、お店が提供するコースについての情報が記入されています。
まず、オーダーメイドプランの価格は「398,000円」です。これは「全コース受講し放題」「フォーラム質問し放題」の他、カリキュラムを受講する際に30分の1